In [3]:
import json
from neo4j import GraphDatabase
from datetime import datetime, timedelta
import random

# Load config from Part 1
with open("healthcare_config.json", "r") as f:
    config = json.load(f)

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [record.data() for record in result]

conn = Neo4jConnection(config["neo4j_uri"], config["neo4j_user"], config["neo4j_password"])
print("Connected to Neo4j!")

Connected to Neo4j!


## 1. Create Hospital and Departments

In [4]:
# Hospital
conn.run_query("""
CREATE (h:Hospital {id: 'H001', name: 'Metro General Hospital', 
        address: '123 Medical Center Dr', totalBeds: 200})
""")

# Departments
departments = [
    {"id": "D001", "name": "ICU", "type": "Critical Care", "capacity": 20},
    {"id": "D002", "name": "Emergency", "type": "Emergency", "capacity": 30},
    {"id": "D003", "name": "Cardiology", "type": "Specialty", "capacity": 25},
    {"id": "D004", "name": "Pediatrics", "type": "General", "capacity": 20},
    {"id": "D005", "name": "Surgery", "type": "Surgical", "capacity": 15},
]

for dept in departments:
    conn.run_query("""
    MATCH (h:Hospital {id: 'H001'})
    CREATE (d:Department $dept)
    CREATE (h)-[:HAS_DEPARTMENT]->(d)
    """, {"dept": dept})

print(f"Created hospital and {len(departments)} departments")

Created hospital and 5 departments


## 2. Create Beds

In [5]:
bed_statuses = ["available", "occupied", "maintenance"]
bed_count = 0

for dept in departments:
    for i in range(dept["capacity"]):
        bed_id = f"B{dept['id'][1:]}{str(i+1).zfill(3)}"
        # 60% occupied, 30% available, 10% maintenance
        status = random.choices(bed_statuses, weights=[30, 60, 10])[0]
        
        conn.run_query("""
        MATCH (d:Department {id: $dept_id})
        CREATE (b:Bed {id: $bed_id, status: $status, floor: $floor})
        CREATE (d)-[:HAS_BED]->(b)
        """, {"dept_id": dept["id"], "bed_id": bed_id, 
              "status": status, "floor": random.randint(1, 5)})
        bed_count += 1

print(f"Created {bed_count} beds")

Created 110 beds


## 3. Create Physicians

In [6]:
physicians = [
    {"id": "PH001", "name": "Dr. Sarah Chen", "specialty": "Critical Care", "dept": "D001", "onCall": True},
    {"id": "PH002", "name": "Dr. James Wilson", "specialty": "Emergency Medicine", "dept": "D002", "onCall": True},
    {"id": "PH003", "name": "Dr. Maria Garcia", "specialty": "Cardiology", "dept": "D003", "onCall": False},
    {"id": "PH004", "name": "Dr. Michael Brown", "specialty": "Pediatrics", "dept": "D004", "onCall": True},
    {"id": "PH005", "name": "Dr. Emily Johnson", "specialty": "Surgery", "dept": "D005", "onCall": False},
    {"id": "PH006", "name": "Dr. David Lee", "specialty": "Critical Care", "dept": "D001", "onCall": False},
    {"id": "PH007", "name": "Dr. Lisa Anderson", "specialty": "Emergency Medicine", "dept": "D002", "onCall": False},
    {"id": "PH008", "name": "Dr. Robert Taylor", "specialty": "Cardiology", "dept": "D003", "onCall": True},
]

for ph in physicians:
    conn.run_query("""
    MATCH (d:Department {id: $dept_id})
    CREATE (p:Physician {id: $id, name: $name, specialty: $specialty, onCall: $onCall})
    CREATE (p)-[:ASSIGNED_TO]->(d)
    """, {"dept_id": ph["dept"], "id": ph["id"], "name": ph["name"], 
          "specialty": ph["specialty"], "onCall": ph["onCall"]})

print(f"Created {len(physicians)} physicians")

Created 8 physicians


## 4. Create Patients

In [7]:
conditions = ["Pneumonia", "Heart Failure", "Fracture", "Appendicitis", 
              "COVID-19", "Stroke", "Diabetes", "Asthma"]
statuses = ["stable", "critical", "recovering", "ready_for_discharge"]

# Get occupied beds
occupied_beds = conn.run_query("""
MATCH (d:Department)-[:HAS_BED]->(b:Bed {status: 'occupied'})
RETURN b.id as bed_id, d.id as dept_id
""")

for i, bed in enumerate(occupied_beds):
    patient_id = f"P{str(i+1).zfill(3)}"
    admit_date = (datetime.now() - timedelta(days=random.randint(1, 14))).isoformat()
    status = random.choice(statuses)
    condition = random.choice(conditions)
    
    conn.run_query("""
    MATCH (b:Bed {id: $bed_id})
    CREATE (p:Patient {id: $patient_id, name: $name, age: $age, 
            status: $status, admitDate: $admit_date, condition: $condition})
    CREATE (p)-[:ADMITTED_TO]->(b)
    """, {"bed_id": bed["bed_id"], "patient_id": patient_id,
          "name": f"Patient {i+1}", "age": random.randint(18, 85),
          "status": status, "admit_date": admit_date, "condition": condition})

print(f"Created {len(occupied_beds)} patients")

Created 71 patients


## 5. Create Inventory

In [8]:
inventory_items = [
    {"id": "I001", "name": "Ventilator", "type": "equipment", "quantity": 15, "minStock": 10, "dept": "D001"},
    {"id": "I002", "name": "Defibrillator", "type": "equipment", "quantity": 8, "minStock": 5, "dept": "D002"},
    {"id": "I003", "name": "IV Bags", "type": "supplies", "quantity": 50, "minStock": 100, "dept": "D001"},
    {"id": "I004", "name": "Surgical Masks", "type": "supplies", "quantity": 200, "minStock": 500, "dept": "D005"},
    {"id": "I005", "name": "Morphine", "type": "medication", "quantity": 30, "minStock": 20, "dept": "D001"},
    {"id": "I006", "name": "Insulin", "type": "medication", "quantity": 45, "minStock": 50, "dept": "D003"},
    {"id": "I007", "name": "Blood Units (O-)", "type": "supplies", "quantity": 12, "minStock": 25, "dept": "D005"},
    {"id": "I008", "name": "Oxygen Tanks", "type": "equipment", "quantity": 20, "minStock": 15, "dept": "D002"},
]

for item in inventory_items:
    low_stock = item["quantity"] < item["minStock"]
    conn.run_query("""
    MATCH (d:Department {id: $dept_id})
    CREATE (i:InventoryItem {id: $id, name: $name, type: $type, 
            quantity: $quantity, minStock: $minStock, lowStock: $lowStock})
    CREATE (d)-[:STOCKS]->(i)
    """, {"dept_id": item["dept"], "id": item["id"], "name": item["name"],
          "type": item["type"], "quantity": item["quantity"], 
          "minStock": item["minStock"], "lowStock": low_stock})

print(f"Created {len(inventory_items)} inventory items")

Created 8 inventory items


## 6. Verify Data

In [9]:
summary = conn.run_query("""
MATCH (h:Hospital)
OPTIONAL MATCH (h)-[:HAS_DEPARTMENT]->(d:Department)
OPTIONAL MATCH (d)-[:HAS_BED]->(b:Bed)
OPTIONAL MATCH (p:Patient)-[:ADMITTED_TO]->(b)
OPTIONAL MATCH (ph:Physician)-[:ASSIGNED_TO]->(d)
OPTIONAL MATCH (d)-[:STOCKS]->(i:InventoryItem)
RETURN h.name as Hospital,
       count(DISTINCT d) as Departments,
       count(DISTINCT b) as Beds,
       count(DISTINCT p) as Patients,
       count(DISTINCT ph) as Physicians,
       count(DISTINCT i) as InventoryItems
""")

print("Database Summary:")
for key, value in summary[0].items():
    print(f"  {key}: {value}")

print("\nData population complete! Proceed to Part 3 for agent definitions.")

Database Summary:
  Hospital: Metro General Hospital
  Departments: 5
  Beds: 110
  Patients: 71
  Physicians: 8
  InventoryItems: 8

Data population complete! Proceed to Part 3 for agent definitions.
